# Rail nodes and navigable water layer must be combined by their most premissive attributes

## North American Rail Nodes

The rail nodes dataset supports the topology of the North American Rail Network (NARN) and attributes information for use in network analysis applications.

Retrieved from https://data-usdot.opendata.arcgis.com/datasets/north-american-rail-nodes/explore?location=37.316236%2C-108.315150%2C4.21 

Retrieved on:  Jan 6, 2022

NTAD datasets are intended for public usage and are not for sale/resale.

See full metadata:  https://data-usdot.opendata.arcgis.com/datasets/usdot::north-american-rail-nodes/about


In [1]:
import os

import rasterio
import numpy as np


In [13]:
data_dir = '/Users/d3y010/projects/cerf/suitability/data'

nodes_shp = os.path.join(data_dir, 'raw', 'North_American_Rail_Nodes', 'North_American_Rail_Nodes.shp')

buffered_shp = os.path.join(data_dir, 'raw', 'North_American_Rail_Nodes', 'North_American_Rail_Nodes_albers_buff5km.shp')

template_raster = os.path.join(data_dir, 'reference', 'cerf_landmask_0-land.tif')

output_temp_raster = os.path.join(data_dir, 'raw', 'North_American_Rail_Nodes','cerf_ntad_railnodes_greaterthan5km_prep.tif')

output_final_raster = os.path.join(data_dir, 'raw', 'North_American_Rail_Nodes','cerf_ntad_railnodes_greaterthan5km.tif')


### In QGIS

1. Reproject to Albers
2. buffer by 5km and dissolve
3. rasterize using the following

In [14]:
# then run this command
gdal_cmd = f'gdal_rasterize -l North_American_Rail_Nodes_albers_buff5km -a BNDRY -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {buffered_shp} {output_temp_raster}'

os.system(gdal_cmd)


0

In [15]:
with rasterio.open(template_raster) as template:
  
    metadata = template.meta
    
    arr_template = template.read(1)
    
    with rasterio.open(output_temp_raster) as opt:
        
        arr = opt.read(1)
        
        arr += arr_template
        
        arr = np.where(arr > 1, 1, arr)

        with rasterio.open(output_final_raster, 'w', **metadata) as dest:
            dest.write(arr, 1)


## Navigable waterway lines

The National Waterway Network is a geographic database of navigable waterways in and around the United States, for analytical studies of waterway performance, for compiling commodity flow statistics, and for mapping purposes.

Retrieved from https://data-usdot.opendata.arcgis.com/datasets/navigable-waterway-lines/explore?location=5.861777%2C-0.334100%2C4.00

Retrieved on:  Jan 6, 2022

NTAD datasets are intended for public usage and are not for sale/resale.

See full metadata:  https://data-usdot.opendata.arcgis.com/datasets/usdot::navigable-waterway-lines/about


In [3]:
import os

import rasterio
import numpy as np
import geopandas as gpd


/Users/d3y010/.pyenv/versions/py3.9.4_cerf/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
data_dir = '/Users/d3y010/projects/cerf/suitability/data'

nodes_shp = os.path.join(data_dir, 'raw', 'Navigable_Waterway_Lines', 'Navigable_Waterway_Lines.shp')

buffered_shp = os.path.join(data_dir, 'raw', 'Navigable_Waterway_Lines', 'Navigable_Waterway_Lines_albers_buff5km.shp')

template_raster = os.path.join(data_dir, 'reference', 'cerf_landmask_0-land.tif')

output_temp_raster = os.path.join(data_dir, 'raw', 'Navigable_Waterway_Lines','cerf_ntad_navwater_greaterthan5km_prep.tif')

output_final_raster = os.path.join(data_dir, 'raw', 'Navigable_Waterway_Lines','cerf_ntad_navwater_greaterthan5km.tif')


### In QGIS

1. Reproject to Albers
2. buffer by 5km and dissolve
3. rasterize using the following

In [6]:
# then run this command
gdal_cmd = f'gdal_rasterize -l Navigable_Waterway_Lines_albers_buff5km -a DIR -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {buffered_shp} {output_temp_raster}'

os.system(gdal_cmd)


0

In [7]:
with rasterio.open(template_raster) as template:
  
    metadata = template.meta
    
    arr_template = template.read(1)
    
    with rasterio.open(output_temp_raster) as opt:
        
        arr = opt.read(1)
        
        arr += arr_template
        
        arr = np.where(arr > 1, 1, arr)

        with rasterio.open(output_final_raster, 'w', **metadata) as dest:
            dest.write(arr, 1)


## Combine

In [19]:
rail_final_raster = os.path.join(data_dir, 'raw', 'North_American_Rail_Nodes','cerf_ntad_railnodes_greaterthan5km.tif')
nav_final_raster = os.path.join(data_dir, 'raw', 'Navigable_Waterway_Lines','cerf_ntad_navwater_greaterthan5km.tif')

combined_raster = '/Users/d3y010/projects/cerf/suitability/data/technology_specific/cerf_ntad_railnodes_navwaters_greaterthan5km.tif'


In [20]:
with rasterio.open(rail_final_raster) as rail:
    
    metadata = rail.meta
    
    arr_rail = rail.read(1)
    
    with rasterio.open(nav_final_raster) as nav:
        
        arr_nav = nav.read(1)
        
        arr_comb = np.minimum(arr_rail, arr_nav)
        
        with rasterio.open(combined_raster, 'w', **metadata) as out:
            out.write(arr_comb, 1)


In [18]:
combined_raster

'/Users/d3y010/projects/cerf/suitability/data/common/cerf_ntad_railnodes_navwaters_greaterthan5km.tif'